## MANIPULACIÓN DE DATOS (SPARK SQL EN DATABRICKS)

In [0]:
from pyspark.sql import SparkSession # Puerta de entrada para trabajar con spark <-- SIEMPRE DEBEMOS IMPORTAR LA LLAVE MAESTRA QUE INICIA TODO.
from pyspark.sql.functions import *  # Funciones propias del módulo SQL de Spark, para trabajar sobre Dataframes.

spark = SparkSession.builder.appName("DatabricksSparkSQL").getOrCreate()

### DATAFRAMES (Versión Databricks Free Edition)

#### Link de [Databrick Free Edition](https://dbc-89f542f8-2df6.cloud.databricks.com/?o=758509963140561)

Para abordar el tema de **Manipulación de Datos**, utilizaremos una de las principales herramientas que ofrece **Databricks: Spark SQL**.  
Esta herramienta nos permite aplicar conceptos del lenguaje SQL sobre conjuntos de datos estructurados, ya sea que provengan directamente de archivos o estén gobernados a través del servicio **Unity Catalog**.

Gracias a Spark SQL, podremos consultar, transformar y analizar datos de forma eficiente, aprovechando tanto la potencia del motor distribuido de Spark como la organización lógica que brinda Unity Catalog.


##### FASE 1. FUENTES DE DATOS

In [0]:
#### ➡️ Como se venía explicando, Unity Catalog trabaja bajo una jerarquía de Catálagos, Esquemas, Volumenes y Tablas.
####     Por ende, es diferente la manera de acceder a su contenido.

#====== 📌 LEER ARCHIVO CSV 
##📝 SINTAXIS:
#   dataset_nombre = spark.read.option("header","true").option("inferSchema","true").csv("/Volumes/NombreCatalago/NombreEsquema/NombreArchivo")

#---> spark.read : Permite leer un archivo basado en una ruta. Siempre instanciando primero SparkSession.
#---> .option("header","true||false") : Permite establecer si tomará en cuenta los encabezados del dataset. true(mostrar) | false(ocultar)
#---> .option("inferSchema","true||false") : Permite reconocer el tipo de dato de cada columna del dataset. true(mostrar) | false(ocultar)
#---> .csv(RutaArchivoCSV) : Tipo de archivo a leer (En este caso un CSV).

## 📝 EJEMPLO:
df_titanic_csv = spark.read.option("header","true").option("inferSchema","true").csv("/Volumes/explicacion_unity_catalog/ejemplos/csv_titanic")
# df_titanic_csv.show(5)

#====== 📌 LEER ARCHIVO PARQUET
##📝 SINTAXIS:
#   dataset_nombre = spark.read.parquet("/Volumes/NombreCatalago/NombreEsquema/NombreArchivoPARQUET")

#---> spark.read : Permite leer un archivo basado en una ruta. Siempre instanciando primero SparkSession.
#---> .parquet(RutaArchivoPARQUET) : Tipo de archivo a leer (En este caso un PARQUET).

## 📝 EJEMPLO
df_titanic_parquet = spark.read.parquet("/Volumes/explicacion_unity_catalog/ejemplos/parquet_titanic")
# df_titanic_parquet.show(5)

#====== 📌 LEER ARCHIVO DELTA
## 📝 SINTAXIS:
#   dataset_nombre = spark.read.format("delta").load("/Volumes/NombreCatalago/NombreEsquema/NombreArchivoDELTA")

#---> spark.read : Permite leer un archivo basado en una ruta. Siempre instanciando primero SparkSession.
#---> .format("delta") : stablece el tipo de archivo a leer (En este caso aplica siempre para DELTA)
#---> .load(RutaArchivoDelta) : Aplica solo a los archivos DELTA.

## 📝 EJEMPLO:
df_titanic_archivo_delta = spark.read.format("delta").load("/Volumes/explicacion_unity_catalog/ejemplos/archivo_delta_titanic")
# df_titanic_archivo_delta.show(5)

#====== 📌 LEER TABLA DELTA (RECOMENDADO Y NATIVO EN DATABRICKS-UNITY-CATALOG)
## 📝 SINTAXIS:
#   dataset_nombre = spark.sql("SELECT * FROM NombreCatalago.NombreEsquema.NombreTablaDelta")

#---> spark.sql() : Permite leer mediante lenguaje SQL una tabla delta. Siempre instanciando primero SparkSession.
#---> "NombreCatalago.NombreEsquema.NombreTablaDelta" : Ruta aplicable solo para las tablas delta y gobernadas por Unity Catalog

## 📝 EJEMPLO
df_titanic_delta = spark.sql("SELECT * FROM explicacion_unity_catalog.ejemplos.delta_titanic")
# df_titanic_delta.show(5)

##### FASE 2. EXPLORACIÓN INICIAL

Utilizaremos de ejemplo el dataset de titanic ( en formato tabla delta y CSV)

In [0]:
#====== A). Quitar encabezados originales provenientes del Dataset (Archivos CSV).
df_titanic_archivo_delta = spark.read.csv("/Volumes/explicacion_unity_catalog/ejemplos/csv_titanic") ## ⬅️ Archivo CSV
df_titanic_archivo_delta.show(3)
#### 🧠💡 Importante: Cuando leamos un archivo CSV, si no establecemos .option("header","true")
####                   el dataframe mostrará las columnas con un prefijo c_ seguido de una posición asignada.
####                   (Y los nombres de columnas serán parte de los registros del dataset)

In [0]:
#====== B). Mostrar los N primeros registros de un dataset.

df_titanic_delta = spark.sql("SELECT * FROM explicacion_unity_catalog.ejemplos.delta_titanic") ## ⬅️ Tabla delta

#### 📝 SINTAXIS: dataset_nombre.show(NúmeroDeRegistros)

#### 📝 EJEMPLO 1:
# df_titanic_delta.show(10) ## ⬅️ Mostrar los 10 primeros registros.

#### 📝 EJEMPLO 2:
# df_titanic_delta.show(5) ## ⬅️ Mostrar los 5 primeros registros.

#### 📝 EJEMPLO 3:
df_titanic_delta.show(3) ## ⬅️ Mostrar los 3 primeros registros.

In [0]:
#====== C). Mostrar los N últimos registros de un dataset (Retornado en una lista).

df_titanic_delta = spark.sql("SELECT * FROM explicacion_unity_catalog.ejemplos.delta_titanic") ## ⬅️ Tabla delta

#### 📝 SINTAXIS: dataset_nombre.tail(NúmeroDeRegistros)

#### 📝 EJEMPLO 1:
# df_titanic_delta.tail(10) ## ⬅️ Mostrar los 10 primeros últimos.

#### 📝 EJEMPLO 2:
# df_titanic_delta.tail(5) ## ⬅️ Mostrar los 5 primeros últimos.

#### 📝 EJEMPLO 3:
df_titanic_delta.tail(3) ## ⬅️ Mostrar los 3 primeros últimos.

In [0]:
#====== D). Mostrar el volúmen del dataset (Cantidad de filas y columnas respectivamente)

df_titanic_delta = spark.sql("SELECT * FROM explicacion_unity_catalog.ejemplos.delta_titanic") ## ⬅️ Tabla delta

#### 📝 SINTAXIS: 
#       dataset_nombre.count() ⬅️ Cantidad de filas
#       len([i for i in dataframe_nombre.columns]) ⬅️ Cantidad de columnas

#### 📝 EJEMPLO 1:
print(f"Cantidad de filas: {df_titanic_delta.count()}") ## ⬅️ Cantidad de filas.

#### 📝 EJEMPLO 2:
print(f"Cantidad de columnas: {len([i for i in df_titanic_delta.columns])}") ## ⬅️ Cantidad de columnas.

##### FASE 3. EXPLORACIÓN INICIAL

###### FASE 3.1. DATOS CUALITATIVOS

In [0]:
"""
    A). CONVERTIR A MAYÚSCULAS LOS DATOS CUALITATIVOS DE UNA COLUMNA EN UN DATASET
    
    📝 SINTAXIS:

        dataframe_nombre = dataframe_nombre.withColumn(
            "NombreColumnaNueva",
            upper(col("NombreColumnaCualitativa")) ⬅️ Función upper() integrada en Spark SQL.
        )
    ### 🧠 En este caso, debemos almacenar en una variable los cambios a realizar en el dataframe
    ### 💡 Importante: Siempre .withColumn() trabajará con al menos una columna del dataset.
"""

# 💡 EJEMPLO:
df_titanic_delta = df_titanic_delta.withColumn(
    "embark_town",
    upper(col("embark_town")) ## ⬅️ Convertimos a mayúsculas los datos de la columna cualitativa ""embark_town""
)
df_titanic_delta.show(3)

In [0]:
"""
    B). CONVERTIR A MINÚSCULAS LOS DATOS CUALITATIVOS DE UNA COLUMNA EN UN DATASET
    
    📝 SINTAXIS:

        dataframe_nombre = dataframe_nombre.withColumn(
            "NombreColumnaNueva",
            lower(col("NombreColumnaCualitativa")) ⬅️ Función lower() integrada en Spark SQL.
        )
    ### 🧠 En este caso, debemos almacenar en una variable los cambios a realizar en el dataframe
    ### 💡 Importante: Siempre .withColumn() trabajará con al menos una columna del dataset.
"""

# 💡 EJEMPLO:
df_titanic_delta = df_titanic_delta.withColumn(
    "embark_town",
    lower(col("embark_town")) ## ⬅️ Convertimos a minúsculas los datos de la columna cualitativa ""embark_town""
)
df_titanic_delta.show(3)

In [0]:
"""
    C). CONVERTIR A MAYÚSCULA LA PRIMERA LETRA DE CADA DATO CUALITATIVOS DE UNA COLUMNA EN UN DATASET
    
    📝 SINTAXIS:

        dataframe_nombre = dataframe_nombre.withColumn(
            "NombreColumnaNueva",
            initcap(col("NombreColumnaCualitativa")) ⬅️ Función initcap() integrada en Spark SQL.
        )
    ### 🧠 En este caso, debemos almacenar en una variable los cambios a realizar en el dataframe
    ### 💡 Importante: Siempre .withColumn() trabajará con al menos una columna del dataset.
"""

# 💡 EJEMPLO:
df_titanic_delta = df_titanic_delta.withColumn(
    "embark_town",
    initcap(col("embark_town")) ## ⬅️ Convertimos la primera letra en mayúscula de cada dato en la columna cualitativa ""embark_town""
)
df_titanic_delta.show(3)

In [0]:
"""
    D). EXTRAER DE CADA DATO CUALITATIVO UNA CADENA EN ESPECÍFICO MEDIANTE >>EXPRESIONES REGULARES<< EN UN DATASET
    
    📝 SINTAXIS:

        dataframe_nombre = dataframe_nombre.withColumn(
            "NombreColumnaNueva",
                regexp_extract(col("embark_town"),r'([A-Z])',1) ⬅️ Función regexp_extract() integrada en Spark SQL.
        )
    ### 🧠 En este caso, debemos almacenar en una variable los cambios a realizar en el dataframe
    ### 💡 Importante: Siempre .withColumn() trabajará con al menos una columna del dataset.
"""

# 💡 EJEMPLO 1: ⬅️ Extraemos primera letra en mayúscula 
# df_titanic_delta = df_titanic_delta.withColumn(
#     "PrimeraLetraEnMayúscula",
#     regexp_extract(col("embark_town"),r'([A-Z])',1)
# )
# df_titanic_delta.show(3)

# 💡 EJEMPLO 2: ⬅️ Extraemos primera letra en minúscula 
# df_titanic_delta = df_titanic_delta.withColumn(
#     "PrimeraLetraEnMinúscula",
#     regexp_extract(col("embark_town"),r'([a-z])',0)
# )
# df_titanic_delta.show(3)

# 💡 EJEMPLO 3: ⬅️ Retornamos todo después de una letra en mayúscula 
# df_titanic_delta = df_titanic_delta.withColumn(
#     "TodoDespuesDeUnaMayúscula",
#     regexp_extract(col("embark_town"),r'[A-Z]([a-z]+)', 1)
# )
# df_titanic_delta.show(3)

# 💡 EJEMPLO 4: ⬅️ Retornamos todo después de una letra en minúscula 
# df_titanic_delta = df_titanic_delta.withColumn(
#     "TodoDespuesDeUnaMinúscula",
#     regexp_extract(col("embark_town"),r'[a-z]([a-z]+)', 1)
# )
# df_titanic_delta.show(3)

# 💡 EJEMPLO 5: ⬅️ Retornamos todo después de un número
# df_titanic_delta = df_titanic_delta.withColumn(
#     "TodoDespuesDeUnNúmero",
#     regexp_extract(col("embark_town"),r'([0-9]+)', 1)
# )
# df_titanic_delta.show(3)


In [0]:
"""
    E). REEMPLAZAR VALORES DE DATOS CUALITATIVOS MEDIANTE >>EXPRESIONES REGULARES<< EN UN DATASET
    
    📝 SINTAXIS:
    
    
    ### 🧠 En este caso, debemos almacenar en una variable los cambios a realizar en el dataframe
    ### 💡 Importante: Siempre .withColumn() trabajará con al menos una columna del dataset.
"""
### DATASET DE PRUEBA PARA ESTE EJEMPLO
datos_sucios = {
    "nombre": [
        "  JUAN  pérez  ", 
        "MARÍA@@LÓPEZ", 
        "Ana--Martínez", 
        "carlos_rodríguez ", 
        "Sofía123 García"
    ],
    "categoria_producto": [
        "ELECTRÓNICA#1", 
        "ropa--Mujer", 
        "hogar_y_DECORACIÓN", 
        "LIBROS@@", 
        "Juguetes  niños"
    ],
    "codigo": [
        "ID-0001", 
        "ID-0023", 
        "CL-1234", 
        "ID-abc123", 
        "id-9999"
    ]
}
## Dataset de prueba (sucio)
df_test = spark.createDataFrame(data=list(zip(*datos_sucios.values())),schema=["nombre","categoria_producto","codigo"])
# df_test.show()

## Dataset de prueba (limpio)
df_test_clean = df_test 
# df_test_clean.show()

## 💡 EJEMPLO 1: ⬅️ Eliminar caractéres especiales (@@,#,--,_)
df_test_clean = df_test_clean.withColumns({
    "categoria_producto":
    regexp_replace(col("categoria_producto"),r'[^A-Za-z0-9ÁÉÍÓÚáéíóúÑñÜü]'," "),
    "nombre":
    regexp_replace(col("nombre"),r'[^A-Za-z0-9ÁÉÍÓÚáéíóúÑñÜü]',' ')
})
# df_test_clean.show()

## 💡 EJEMPLO 2: ⬅️ Normalizar espacios (Varios espacios en blanco a uno solo)
df_test_clean = df_test_clean.withColumns({
    "categoria_producto":
    regexp_replace(col("categoria_producto"),r'\s+'," "),
    "nombre":
    regexp_replace(col("nombre"),r'\s+',' ')
})
# df_test_clean.show()

## 💡 EJEMPLO 3: ⬅️ Reemplazar guiones por espacios en blanco
df_test_clean = df_test_clean.withColumn(
    "nombre",
    regexp_replace(col("nombre"),r'[-_]',' ')
)
# df_test_clean.show()

## 💡 EJEMPLO 4: ⬅️ Eliminar prefijos (ID- || CL- || id- || Letras)
df_test_clean = df_test_clean.withColumn(
    "codigo",
    regexp_replace(col("codigo"),r'^ID-|^CL-|^id-|[a-zA-Z]','')
)
# df_test_clean.show()

## 💡 EJEMPLO 5: ⬅️ Eliminar números dentro de cadenas
df_test_clean = df_test_clean.withColumns({
    "nombre":
    regexp_replace(initcap(col("nombre")),r'[0-9]',''),
    "categoria_producto":
    regexp_replace(initcap(col("categoria_producto")),r'[0-9]','')
})
df_test_clean.show()


###### FASE 3.2. DATOS CUANTITATIVOS

In [0]:
df_penguins = spark.sql("SELECT * FROM workspace.exercises.penguins")
df_penguins.show(3)

In [0]:
"""
    A). EXPLORACIÓN BÁSICA Y RESUMEN ESTADÍSTICO
        
        📝 Para los datos cuantitativos podemos obtener su estadística básica
            mediante .decribe() el cuál retornará el valor de las columnas float o int.
            En los campos cualitativos se visualizará información no creíble.
"""
df_penguins.describe().show()                            ## ⬅️ Estadística simple - Todas las columnas
df_penguins.select(col("body_mass_g")).describe().show() ## ⬅️ Estadística simple - Columna body_mass_g
df_penguins.select(mean(col("body_mass_g"))).show()      ## ⬅️ Media estadística - Columna body_mass_g
df_penguins.select(median(col("body_mass_g"))).show()    ## ⬅️ Mediana estadística - Columna body_mass_g
df_penguins.select(min(col("body_mass_g"))).show()       ## ⬅️ Valor mínimo - Columna body_mass_g
df_penguins.select(max(col("body_mass_g"))).show()       ## ⬅️ Valor máximo (IMC Gramos)
df_penguins.select(sum(col("body_mass_g"))).show()       ## ⬅️ Suma total - Columna body_mass_g
df_penguins.select(stddev(col("body_mass_g"))).show()    ## ⬅️ Desviación estándar - Columna body_mass_g
df_penguins.select(variance(col("body_mass_g"))).show()  ## ⬅️ Varianza - Columna body_mass_g



In [0]:
"""
    B). VERIFICAR VALORES NULOS EN LA(S) COLUMNA(AS) DEL DATASET
"""

## 💡 EJEMPLO 1: VERIFICAR CANTIDAD DE VALORES NULOS EN TODAS LAS COLUMNAS DEL DATASET
## ⬅️ Retorna la cantidad de datos nulos por columna.
cantidad_nulos = df_penguins.select([
    sum(when(col(i).isNull(),1).otherwise(0)).alias(i)
    for i in df_penguins.columns
])
# cantidad_nulos.show()

## 💡 EJEMPLO 2: VERIFICAR CANTIDAD DE VALORES NULOS EN UNA COLUMNA ESPECÍFICA DEL DATASET
##⬅️ Retorna la cantidad de datos nulos de una columna.
cantidad_nulos_bmg = df_penguins.select([
    sum(when(col(i).isNull(),1).otherwise(0)).alias(i)
    for i in df_penguins.columns
    if i=="body_mass_g"
])
cantidad_nulos_bmg.show()

In [0]:
"""
    C). FILTRANDO VALORES EN COLUMNAS CUANTITATIVAS
"""

print("Total de datos registrados: ",df_penguins.count()) ## ⬅️ Cantidad de datos del dataset : 344

## 💡 EJEMPLO 1: FILTRAR VALORES DE COLUMNA "body_mass_g" MAYOR A 3000
df_penguins_bmg_mayor_3000 = df_penguins.filter(
    col("body_mass_g")>3000
)
# df_penguins_bmg_mayor_3000.show(3)
print("Total de datos filtro 1: ",df_penguins_bmg_mayor_3000.count()) ## ⬅️ Cantidad de datos del dataset : 331

## 💡 EJEMPLO 2: FILTRAR VALORES DE COLUMNA "body_mass_g" ENTRE 1000 y 3000
df_penguins_bmg_entre_1000_3000 = df_penguins.filter(
    (col("body_mass_g")>=1000) &
    (col("body_mass_G")<=3000)
)
# df_penguins_bmg_entre_1000_3000.show(3)
print("Total de datos filtro 2: ",df_penguins_bmg_entre_1000_3000.count()) ## ⬅️ Cantidad de datos del dataset : 11

## 💡 EJEMPLO 3: FILTRAR VALORES NULOS EN COLUMNA "body_mass_g"
df_penguins_nulos_body_mass_g = df_penguins.filter(
    col("body_mass_g").isNull()
)
# df_penguins_nulos_body_mass_g.show(3)
print("Total de datos filtro 3: ",df_penguins_nulos_body_mass_g.count()) ## ⬅️ Cantidad de datos del dataset : 2

## 💡 EJEMPLO 4: FILTRAR VALORES NO NULOS EN COLUMNA "body_mass_g"
df_penguins_no_nulos_body_mass_g = df_penguins.filter(
    col("body_mass_g").isNotNull()
)
# df_penguins_no_nulos_body_mass_g.show(3)
print("Total de datos filtro 4: ",df_penguins_no_nulos_body_mass_g.count()) ## ⬅️ Cantidad de datos del dataset : 342


In [0]:
"""
    D). RELLENANDO DATOS Null DE LAS COLUMNAS DE UN DATASET
    📝 SINTAXIS:
        df_penguins = df_penguins.fillna(value=ValorARellenar,subset=NombreColumna)
        ### 🧠 Importante: En este caso, los valores null deben rellenarse con datos
                            del mismo tipo que la columna. Por ende, debemos llenar 
                            individualmante.
"""
### ➡️ Verificar datos nulos
cantidad_nulos = df_penguins.select([
    sum(when(col(i).isNull(),1).otherwise(0)).alias(i)
    for i in df_penguins.columns
])
# cantidad_nulos.show()

### ➡️ Verificar datos nulos que han sido rellenados.
df_penguins = df_penguins.fillna(value=15.55,subset=["body_mass_g"])
# df_penguins.show()

cantidad_nulos.show() ##  ✅Datos nulos rellanados correctamente.


##### FASE 4. AGRUPAMIENTO DE INFORMACIÓN

El agrupamiento de la información nos brinda el resumen de la misma proveniente 
de un dataset, logrando encontrar ciertos patrones en cada grupo de información

In [0]:
"""
    📝 SINTAXIS:

        dataset_nuevo_agrupamiento= dataset_nombre.groupBy(["NombreColumna1","NombreColumna2]).agg(
            funciónAgregación(col("NombreColumna")).alias("NuevoNombreColumna")
        )
        
        ### 🧠 En este caso, debemos almacenar en una variable los cambios a realizar en el dataframe.

❎ En el agrupamiento de información, podemos utilizar diversas funciones de agregación, tales como:

💡.min():    ⬅️ Permite obtener el mínimo valor de la información agrupada.
💡.max():    ⬅️ Permite obtener el máximo valor de la información agrupada.
💡.sum():    ⬅️ Permite sumar la información de una columna cuantitativa por la información agrupada.
💡.count():  ⬅️ Permite contar la cantidad de información de una columna por la información agrupada.
💡.mean():   ⬅️ Permite obtener la media de una columna cuantitativa por la información agrupada.
💡.median(): ⬅️ Permite obtener la mediana de una columna cuantitativa por la información agrupada.
"""
###✔️ Usaremos el dataset de "penguins"
df_penguins = spark.sql("SELECT * FROM workspace.exercises.penguins") ## ⬅️ Tabla Delta
# df_penguins.show() 

In [0]:
## 💡 EJEMPLO 1: (AGRUPANDO POR UNA COLUMNA) 
df_agrupado_uno = df_penguins.groupBy(["species"]).agg(
    round(mean(col("body_mass_g")),2).alias("PromedioBMG") ## ⬅️ Hallamos la media de la columna "body_mass_g"
)
# df_agrupado_uno.show()

## 💡 EJEMPLO 2: (AGRUPAR POR DOS COLUMNAS)
df_agrupado_dos = df_penguins.groupBy(["species","sex"]).agg(
    round(mean(col("body_mass_g")),2).alias("Species_Sex_PromedioBMG")
)
# df_agrupado_dos.show()

## 💡 EJEMPLO 3: (AGRUPAR LA INFORMACIÓN POR LA CANTIDAD DE LA MISMA COLUMNA)

df_agrupado_tres = df_penguins.groupBy(["species"]).agg(
    count(col("species")).alias("Cantidad") ## ⬅️ .count() : función clave para traer la cantidad de datos de la columna.
)
df_agrupado_tres.show()